In [1]:
!pip install langchain pypdf chromadb openai tiktoken python-dotenv

Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 9.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/19.8 MB ? eta -:--:--
   ----- ---------------------------------- 2.6/19.8 MB 15.1 MB/s eta 0:00:02
   ------------ --------------------------- 6.3/19.8 MB 16.1 MB/s eta 0:00:01
   ---------------- ----------------------- 8.1/19.8 MB 13.2 MB/s eta 0:00:01
   -------------------- ------------------- 10.0/19.8 MB 12.4 MB/s eta 0:00:01
   ----------------------- ---------------- 11.5/19.8 MB 11.5 MB

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress t

In [7]:
pip install -U langchain-community pypdf

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ----------------------------- ---------- 1.8/2.5 MB 10.1 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 11.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [17]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("../policies/bcim-policy.pdf")
docs = loader.load()
len(docs), docs[0].page_content[:500]

(6,
 'Page 1 of 6 \n \nBusiness Continuity and Incident Management Policy \n \n1 Introduction \n \n1.1 The International Standard ISO22301  defines Business Continuity as ‘The capability of an \norganisation to continue delivery of products or services at acceptable predefined levels  \nfollowing a disruption’. \n1.2 The University of London is committed to protecting the safety and welfare of all interested \nparties, to the continued delivery of key operations and return to business as usual as quickly \nas po')

In [19]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = splitter.split_documents(docs)
len(split_docs), split_docs[0].page_content

(16,
 'Page 1 of 6 \n \nBusiness Continuity and Incident Management Policy \n \n1 Introduction \n \n1.1 The International Standard ISO22301  defines Business Continuity as ‘The capability of an \norganisation to continue delivery of products or services at acceptable predefined levels  \nfollowing a disruption’. \n1.2 The University of London is committed to protecting the safety and welfare of all interested \nparties, to the continued delivery of key operations and return to business as usual as quickly \nas possible, following a disruptive incident. \n1.3 The University will implement and maintain an  effective Business Continuity and Incident \nManagement (BCIM) programme to support the strategic objectives of the organisation and \nproactively build the capability to continue business operations in the event of disruption. \nThis includes the identification of risks and threats; the creation of response structures and')

In [13]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = OpenAIEmbeddings()
vectordb = Chroma.from_documents(split_docs, embeddings, persist_directory="db_test/")
vectordb.persist()

C:\Users\asus\AppData\Local\Temp\ipykernel_20956\1199919350.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()
C:\Users\asus\AppData\Local\Temp\ipykernel_20956\1199919350.py:6: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [14]:
query = "Who is responsible for Business Continuity at the University of London?"
docs = vectordb.similarity_search(query, k=2)
for d in docs:
    print(d.page_content[:300])

4.8 To ensur e plans are subject  to regular review , audit and exercise, adopted across the 
University and that Business Continuity is embedded into the culture of the University  so 
that it becomes an integral part of decision making. 
4.9 To raise the profile of BCIM within the University to in
Page 4 of 6 
 
The remit of this  organisation-wide group is to oversee the development  of Business 
Continuity and Incident Management Planning on behalf of the Business Continuity Lead.  
• The group is chaired by the Business Continuity Manager  and includes 
representatives across the Universit


In [30]:
# with FAISS
!pip install faiss-cpu

Defaulting to user installation because normal site-packages is not writeable


In [28]:
# Imports
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

# Replace Chroma with FAISS
embeddings = OpenAIEmbeddings()
vectordb = FAISS.from_documents(split_docs, embeddings)

# Save FAISS index to disk
vectordb.save_local("faiss_index")

In [29]:
# Run a query against FAISS index
query = "Who is responsible for Business Continuity at the University of London?"
docs = vectordb.similarity_search(query, k=2)

for d in docs:
    print(d.page_content[:300])  # show first 300 chars of each match


4.8 To ensur e plans are subject  to regular review , audit and exercise, adopted across the 
University and that Business Continuity is embedded into the culture of the University  so 
that it becomes an integral part of decision making. 
4.9 To raise the profile of BCIM within the University to in
Page 4 of 6 
 
The remit of this  organisation-wide group is to oversee the development  of Business 
Continuity and Incident Management Planning on behalf of the Business Continuity Lead.  
• The group is chaired by the Business Continuity Manager  and includes 
representatives across the Universit


In [21]:
pip install -U langchain-openai langchain-community

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/948.4 kB ? eta -:--:--
   --------------------------------------- 948.4/948.4 kB 10.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.76.1
    Uninstalling openai-1.76.1:
      Successfully uninstalled openai-1.76.1
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [22]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

In [23]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [24]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(),
    chain_type="stuff"  # simplest type, just stuffs chunks into prompt
)

In [25]:
query = "Who is responsible for Business Continuity at the University of London?"
answer = qa_chain.run(query)
print(answer)

C:\Users\asus\AppData\Local\Temp\ipykernel_20956\52612677.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa_chain.run(query)


At the University of London, the Vice-Chancellor is the owner and senior stakeholder for Business Continuity. The Pro Vice-Chancellor (Finance and Operations) is the lead for Business Continuity on behalf of the Vice-Chancellor. The Business Continuity Manager, who is the Associate Director IT Security and Business Continuity, is responsible for managing the implementation of the BCIM Policy and Programme on a day-to-day basis.


In [26]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(),
    chain_type="stuff",
    return_source_documents=True
)

query = "Who is responsible for Business Continuity at the University of London?"
result = qa_chain({"query": query})

print("Answer:")
print(result["result"])
print("\nSources:")
for doc in result["source_documents"]:
    print(f"- {doc.metadata.get('source')} | Preview: {doc.page_content[:200]}...")


C:\Users\asus\AppData\Local\Temp\ipykernel_20956\2383957659.py:11: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": query})


Answer:
At the University of London, the Vice-Chancellor is the owner and senior stakeholder for Business Continuity. The Pro Vice-Chancellor (Finance and Operations) is the lead for Business Continuity on behalf of the Vice-Chancellor. The Business Continuity Manager, who is the Associate Director IT Security and Business Continuity, is responsible for managing the implementation of the BCIM Policy and Programme on a day-to-day basis.

Sources:
- ../policies/bcim-policy.pdf | Preview: 4.8 To ensur e plans are subject  to regular review , audit and exercise, adopted across the 
University and that Business Continuity is embedded into the culture of the University  so 
that it become...
- ../policies/bcim-policy.pdf | Preview: Page 4 of 6 
 
The remit of this  organisation-wide group is to oversee the development  of Business 
Continuity and Incident Management Planning on behalf of the Business Continuity Lead.  
• The gro...
- ../policies/bcim-policy.pdf | Preview: Page 3 of 6 
 
The 